# <center> Workshop - Intro to MDAnalysis Part 2</center>


# Distance calculations in MDAnalysis

Atom coordinates are in the 
`.positions` attribute of an `AtomGroup`

The positions are returned as a NumPy array, which we can then readily manipulate.




In [ ]:
# First we import MDAnalysis
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import GRO, TRR

u = mda.Universe(GRO, TRR)

ag = u.select_atoms('protein')

pos = ag.positions

print(f'AtomGroup has length {len(ag)} and positions is shape {pos.shape}')

pos

Some built-in functions based on position data:

- `center_of_mass()`

- `center_of_geometry()`


In [ ]:
print(ag.center_of_mass())

print(ag.center_of_geometry())

## The `lib.distances` module

Particle positions are given as numpy arrays, so most work can be done using numpy (and numpy derived) libraries.

MDAnalysis  `lib.distances` module comes in handy when

- we have periodic boundary conditions (numpy cannot handle this)

- domain specific algorithms can be used


In [ ]:
from MDAnalysis.lib import distances

distances

### `distance_array`

To calculate **all** pairwise distances between two arrays of coordinates.

In [ ]:
ag1 = u.atoms[:10]
ag2 = u.atoms[10:30]


da = distances.distance_array(ag1.positions, 
                              ag2.positions,
                              box=u.dimensions)

print(f'Our input atomgroups has sizes {len(ag1)} and {len(ag2)}, the output has shape: {da.shape}')
print()

print(da)

The output of distance array is a matrix of the distance between each position in the first coordinate array and each position in the second coordinate array.

For example to look at the distance between the 4th and 6th atom in the two AtomGroups:

In [ ]:
print(f'The distance between {ag1[3]} and {ag2[5]} is {da[3, 5]} A')

### `self_distance_array`

For calculating distances between all combinations of coordinates.

Takes a **single array** of coordinates and calculates all pairwise distances.
This will yield  ½ n(n-1) distances.


In [ ]:
sda = distances.self_distance_array(ag1.positions, box=None)

print(f'Our input AtomGroup has size {len(ag1)} and the output has shape {sda.shape}')


### `calc_bonds`

For calculating a series of distances between pairs of coordinates.

Takes 2 arrays of coordinates **of equal length**, and returns the distances between coordinate pairs in each row.


In [ ]:
coords1 = u.atoms[:10].positions
coords2 = u.atoms[10:20].positions
dist = distances.calc_bonds(coords1, 
                            coords2, 
                            box=None)

print(f'The inputs has length {len(coords1)} and {len(coords2)} and the output has shape {dist.shape}')
print()
print(f'The distance between the first coordinate in each array is: {dist[0]}')

### `calc_angles` & `calc_dihedrals`

Calculates either the angle or dihedral angle between 3 or 4 arrays of coordinates.
Takes 3 or 4 arrays of **identical length** coordinates.

For angles, the middle array is the apex of the angle.

For dihedrals, the angle is formed between the plane of the first three coordinates, and the plane of the second three coordinates.


In [ ]:
import numpy as np
coords1 = u.atoms[:10].positions
coords2 = u.atoms[10:20].positions
coords3 = u.atoms[20:30].positions

angles = distances.calc_angles(
            coords1, coords2, coords3,
            box=None, result=None)

print(np.rad2deg(angles))


coords4 = u.atoms[30:40].positions

dihedrals = distances.calc_dihedrals(coords1, coords2, coords3, coords4)

# Minimum image convention

To account for periodic boundary conditions in distance calculations,
pass the box information as `box=ag.dimensions` to any distance or angle function.

This makes the distance calculation take the minimum image convention into account when calculating distance,
which makes the measured distances equal to the minimum possible between all periodic images of the two coordinates.

In [ ]:
print(f'The box size of our Universe is {u.dimensions}')

In [ ]:
protein = u.select_atoms('protein')

print(f'The maximum distance without periodic boundaries is {distances.self_distance_array(protein).max()}')

print(f'The maximum distance with periodic boundaries is {distances.self_distance_array(protein, box=u.dimensions).max()}')

# `capped_distance` and `self_capped_distance`

Only find distances up to a maximum limit. It returns:
- an array of indices
- an array of distances

This is much more efficient when dealing with large (>50,000 atoms) systems.

For example, the start of a hydrogen bond analysis might look like:

In [ ]:
hydrogens = u.select_atoms('resname SOL and type H')
acceptor = u.select_atoms('protein and type N O')

print(f'We have {len(hydrogens)} hydrogens and {len(acceptor)} acceptors')

ix, dist = distances.capped_distance(hydrogens.positions, 
                          acceptor.positions, 
                          min_cutoff =1.0,
                          max_cutoff=4.0,
                          box=u.dimensions)

print(f'We found {len(ix)} contacts less then 4.0 A')
print()
print(f'The first three are {ix[:3]} at distances {dist[:3]}')

We can see that capped_distance is approximately 10x faster than the brute force solution.

In [ ]:
%timeit distances.distance_array(hydrogens.positions, acceptor.positions, box=u.dimensions)

In [ ]:
%timeit distances.capped_distance(hydrogens.positions, acceptor.positions, min_cutoff=1.0, max_cutoff=4.0, box=u.dimensions)

# A summary of Lecture 2

Calculating pairwise distances:
- calc_bonds
- distance_array
- self_distance_array

Faster, sparse pairwise distances:
- capped_distance
- self_capped_distance

Calculating angles:
- calc_angles
- calc_dihedrals

Use `box=u.dimensions` to take minimum image convention into account (if you want to!).

2022-07-26 11:19:45 2022-07-26 11:19:46 ## A summary of Lecture 1

Most simulation analysis will involve extracting position data from certain atoms.

- A `Universe` contains all information about a simulation system

- An `AtomGroup` contains information about a group of atoms

- We can use `Universe.select_atoms()` to create an `AtomGroup` containing specific atoms from a `Universe`

- Positions of atoms in an AtomGroup are accessed through `AtomGroup.positions`